# FLDAS

In [ ]:
# Import relevant libraries
import ee
import geemap
import pandas as pd

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-lscogill4')

In [ ]:
# define region with point coords
region = ee.Geometry.Point([29.23397, -28.994639]) # coords of the flux tower to go here

# Define the date range
start_date = '2014-06-01'
end_date = '2023-09-01'

# Load FLDAS ET monthly data from Earth Engine
fldas = ee.ImageCollection("NASA/FLDAS/NOAH01/C/GL/M/V001")

# Filter the FLDAS dataset by region and date
filtered_fldas = fldas.filterBounds(region).filterDate(start_date, end_date)

In [ ]:
# Define a function to acquire monthly ET

# Only 1 ET value per month for this product (FLDAS timeseries comes in a monthly timestep already),
# Therefore no mean or sums to be calculated
def calculate_monthly_et(image_collection, date_str):
    """Calculates the monthly sum of ET for a given image collection and date.

    Args:
        image_collection (ee.ImageCollection): The image collection to process.
        date_str (str): The date string in 'YYYY-MM' format.

    Returns:
        dict: A dictionary containing the date and monthly ET sum.
    """
    monthly_fldas_et = image_collection.select('Evap_tavg').first().reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=region,
        scale=11132,
        maxPixels=1e9
    )
    return {
        'Date': date_str,
        'Monthly_ET': monthly_fldas_et.get('Evap_tavg').getInfo()
    }

# Define a function to calculate monthly sums
def calculate_monthly(collection):
    """Calculates monthly sums of ET for the given collection.

    Args:
        collection (ee.ImageCollection): The image collection to process.

    Returns:
        list: A list of dictionaries containing the date and monthly ET sum for each month.
    """
    start = ee.Date(start_date)
    end = ee.Date(end_date)
    months = ee.List.sequence(0, end.difference(start, 'months').subtract(1))
    results = []

    def monthly_composite(month_offset):
        month_start = start.advance(month_offset, 'months')
        month_end = month_start.advance(1, 'months')
        date_str = month_start.format('YYYY-MM').getInfo()
        monthly_images = collection.filterDate(month_start, month_end)
        et_result = calculate_monthly_et(monthly_images, date_str)
        results.append(et_result)

    for month_offset in months.getInfo():
        monthly_composite(month_offset)

    return results

# Extract monthly ET (ET still in kg/m2/s and needs to be converted to mm/month in the next block of code below)
monthly_et_fldas_results = calculate_monthly(filtered_fldas)

# Convert the results to a DataFrame
monthly_fldas_et_df = pd.DataFrame(monthly_et_fldas_results)

# Display the DataFrame
print(monthly_fldas_et_df)

In [ ]:
# Import relevant libraries
import pandas as pd
from calendar import monthrange

# Convert the 'Date' column to datetime objects
monthly_fldas_et_df['Date'] = pd.to_datetime(monthly_fldas_et_df['Date'], format='%Y-%m')

# Function to get the number of days in the month for each date
monthly_fldas_et_df['Days_in_Month'] = monthly_fldas_et_df['Date'].apply(lambda date: monthrange(date.year, date.month)[1])

# Multiply Monthly_ET by the number of days in each month and by 86400 for conversion [*]
# [*Conversion: ET kg/m2/s1 = x 86400 (24*60*60) x number of days in each month = ET mm/month]
monthly_fldas_et_df['Monthly_ET'] = monthly_fldas_et_df['Monthly_ET'] * monthly_fldas_et_df['Days_in_Month'] * 86400

# Display the updated DataFrame
print(monthly_fldas_et_df)

# Convert the result to a list to export it as a DataFrame
et_fldas_list = monthly_fldas_et_df['Monthly_ET'].tolist()
print(et_fldas_list)
dates_fldas_list = monthly_fldas_et_df['Date'].tolist()
print(dates_fldas_list)

In [ ]:
# Combine dates and ET data into a DataFrame
et_fldas_df = pd.DataFrame({
    'date': dates_fldas_list,
    'evapotranspiration': et_fldas_list
})

#  Export the DataFrame to CSV
et_fldas_df.to_csv('monthly_fldas_data.csv', index=False)
print("CSV file has been created.")